In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-19 04:32:23--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.42MB/s    in 0.2s    

2022-08-19 04:32:23 (4.42 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
# Start Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Automotive").getOrCreate()

# Load Dataset to Dataframe

In [ ]:
# Load in user_data.csv from S3 into a DataFrame
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), inferSchema=True, sep="\t")

df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [ ]:
# Count the rows
df.count()

3514942

In [ ]:
# Drop Null Values
df = df.dropna()

In [ ]:
# Count the rows
df.count()

3514045

In [ ]:
# Drop Duplicates
df = df.dropDuplicates()

In [ ]:
# Count the rows
df.count()

3514045

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Change Datatype of review_date
df = df.withColumn("review_date",df.review_date.cast('date'))

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



#Transform Dataframe to Fit the Tables in the Schema Files

##Create review_id Table

In [ ]:
# Create review_id Dataframe
review_id_df = df.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100B3Y924RGIH|   14305020|B00B0GCZ3I|     881172530| 2014-05-29|
|R100BI6X9TR7G8|   11530488|B00CC2ZU4K|     954296244| 2015-01-21|
|R100IKQF3ORNJF|    9900991|B0091RCKPG|     240957722| 2014-01-10|
|R100PNUOGPNLHB|   22085873|B00EC4OUT2|     726003134| 2015-08-26|
|R100QSKTYYHYBD|   21945587|B001PF4ZZ0|     155818034| 2015-07-31|
|R100RIURK0DUFM|     659629|B000MSDKMA|     622787089| 2015-02-16|
|R100SPLJRY8XB2|    6733934|B005U30JXK|     937150161| 2015-08-11|
|R101691UN8LFDK|   21003951|B004DESMEK|      62726443| 2013-06-04|
|R1018Z926QMNB9|   41066228|B00EECM5FI|     667684414| 2015-01-26|
|R101OY4J4NHTGH|   52526844|B005VIIVZW|     211604382| 2014-04-17|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [ ]:
# View Datatype for review_id_df
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



##Create products Table

In [ ]:
# Create products Dataframe
products_df = df.select(["product_id","product_title"])
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00B0GCZ3I|Seeutek PK80 80cc...|
|B00CC2ZU4K|OPT7 Bolt AC 55w ...|
|B0091RCKPG|Peerless 0231705 ...|
|B00EC4OUT2|Andoer 2x White U...|
|B001PF4ZZ0|Dynatek Ignition ...|
|B000MSDKMA|Optima Batteries ...|
|B005U30JXK|vent window shade...|
|B004DESMEK|Genuine Honda 08U...|
|B00EECM5FI|HID Xenon Low Bea...|
|B005VIIVZW|Genuine GM Parts ...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
# View Datatype for products 
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



##Create customers Table

In [ ]:
# Create customers Dataframe
customers_df = df.groupBy("customer_id").count()
customers_df.show(10)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   38403718|    7|
|   32746205|    1|
|   18898952|    3|
|   14985245|    1|
|   28988871|    3|
|   37690053|    5|
|   10357929|   12|
|   10455779|    2|
|   48376350|    3|
|    9448435|   11|
+-----------+-----+
only showing top 10 rows



In [ ]:
# Rename count Column
customers_df = customers_df.withColumnRenamed("count","customer_count")
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   38403718|             7|
|   32746205|             1|
|   18898952|             3|
|   14985245|             1|
|   28988871|             3|
|   37690053|             5|
|   10357929|            12|
|   10455779|             2|
|   48376350|             3|
|    9448435|            11|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# View Datatype for customers 
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



##Create vine_table

In [ ]:
# Create vine_table Dataframe
vine_table_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_table_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100B3Y924RGIH|          1|            8|         10|   N|
|R100BI6X9TR7G8|          5|            0|          1|   N|
|R100IKQF3ORNJF|          5|            0|          2|   N|
|R100PNUOGPNLHB|          4|            0|          0|   N|
|R100QSKTYYHYBD|          5|            0|          1|   N|
|R100RIURK0DUFM|          5|            0|          0|   N|
|R100SPLJRY8XB2|          1|            0|          0|   N|
|R101691UN8LFDK|          3|            1|          1|   N|
|R1018Z926QMNB9|          5|           15|         18|   N|
|R101OY4J4NHTGH|          1|            8|          8|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
# View Datatype for vine_table_df
vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



#Load Dataframe to RDS

In [ ]:
# Configure for RDS
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>.rds.amazonaws.com:5432/automotive_db"
config = {"user":"root",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to review_id_df table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine_table table in RDS

vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)